In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from utils import *
from sqlparse import format

In [2]:
from django.db.models import (
    Q,
    F,
    Case,
    When,
    Count,
    Func,
    Min,
    Max,
    Sum,
    Avg,
    Value,
    OuterRef,
    Subquery,
    CharField,
)
from django.db.models.functions import Concat, Cast, Round, Length
from django.db import connection
from customer_db.models import Provinces, Patients, Doctors, Admissions


In [3]:
def print_sql(queryset):
    formatted = format(str(queryset.query), reindent=True)
    print(formatted)

## Testing some basic queries

In [4]:
sqlq = sql_raw("SELECT * FROM Patients limit 3")
sql_to_df(sqlq)

,patient_id,first_name,last_name,gender,birth_date,city,allergies,height,weight,province_id
0,1,Cheryl,Mason,F,1978-05-14,Toronto,Peanuts,170,65,ON
1,2,Jason,Fleming,M,1990-08-02,Vancouver,Sulfa drugs,185,80,BC
2,3,Tiffany,McDonald,F,1985-03-27,Calgary,Lactose,155,58,AB


In [5]:
common_fields = ['first_name','last_name','gender',
        'birth_date','city','province_id','allergies','height','weight']

In [6]:
ormq = Patients.objects\
.values(patient_id=F('id'), *common_fields)[:3]
orm_to_df(ormq)

,first_name,last_name,gender,birth_date,city,province_id,allergies,height,weight,patient_id
0,Cheryl,Mason,F,1978-05-14,Toronto,ON,Peanuts,170,65,1
1,Jason,Fleming,M,1990-08-02,Vancouver,BC,Sulfa drugs,185,80,2
2,Tiffany,McDonald,F,1985-03-27,Calgary,AB,Lactose,155,58,3


In [7]:
equal(sqlq, ormq)

Equal ✔️


## Easy 1
Show first name, last name, and gender of patients who's gender is 'M'

In [8]:
qstr = """
SELECT
  first_name,
  last_name
FROM Patients
WHERE gender = 'M';
"""
sqlq = sql_raw(qstr)

In [9]:
ormq =  Patients.objects.filter(gender='M').values('first_name', 'last_name')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name"
FROM "patients"
WHERE "patients"."gender" = M


In [10]:
orm_to_df(ormq[:3])

,first_name,last_name
0,Jason,Fleming
1,Alex,Carter
2,Gavin,Bryant


## Easy 2
Show first name and last name of patients who does not have allergies. (null)

In [11]:
qstr = """
SELECT
  first_name,
  last_name
FROM Patients
WHERE allergies IS NULL;
"""
sqlq = sql_raw(qstr)

In [12]:
ormq =  Patients.objects.filter(allergies=None).values('first_name', 'last_name') 
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name"
FROM "patients"
WHERE "patients"."allergies" IS NULL


In [13]:
ormq1 =  Patients.objects.filter(allergies__isnull=True).values('first_name', 'last_name')
equal(sqlq, ormq1)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name"
FROM "patients"
WHERE "patients"."allergies" IS NULL


In [14]:
orm_to_df(ormq[:3])

,first_name,last_name
0,Alex,Carter
1,Adam,Nguyen
2,Shane,Simpson


## Easy 3
Show first name of patients that start with the letter 'C'

In [15]:
qstr = """
SELECT first_name
FROM Patients
WHERE first_name LIKE 'C%';
"""
sqlq = sql_raw(qstr)

In [16]:
ormq =  Patients.objects.filter(first_name__startswith='C').values('first_name')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name"
FROM "patients"
WHERE "patients"."first_name" LIKE C% ESCAPE '\'


In [17]:
orm_to_df(ormq[:3])

,first_name
0,Cheryl
1,Cassandra
2,Cameron


## Easy 4
Show first name and last name of patients that weight within the range of 100 to 120 (inclusive)
We will do 70 - 80

In [18]:
qstr = """
SELECT
  first_name,
  last_name
FROM Patients
WHERE weight BETWEEN 70 and 80;
"""
sqlq = sql_raw(qstr)

In [19]:
ormq =  Patients.objects.filter(weight__range=[70, 80]).values('first_name', 'last_name')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name"
FROM "patients"
WHERE "patients"."weight" BETWEEN 70 AND 80


In [20]:
ormq1 =  Patients.objects.filter(weight__gte=70, weight__lte=80).values('first_name', 'last_name')
equal(sqlq, ormq1)
print_sql(ormq1)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name"
FROM "patients"
WHERE ("patients"."weight" >= 70
       AND "patients"."weight" <= 80)


In [21]:
orm_to_df(ormq[:3])

,first_name,last_name
0,Jason,Fleming
1,Alex,Carter
2,Gavin,Bryant


## Easy 5
Show first name and last name concatinated into one column to show their full name.

In [22]:
qstr = """
SELECT
  first_name || ' ' || last_name full_name
FROM patients;
"""
sqlq = sql_raw(qstr)

In [23]:
# sqlq =  sql_raw("SELECT CONCAT(first_name,' ', last_name) full_name FROM Patients;")

ormq =  Patients.objects.values(full_name=Concat('first_name', Value(' '), 'last_name'))
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT COALESCE("patients"."first_name",) || COALESCE(COALESCE(,) || COALESCE("patients"."last_name",),) AS "full_name"
FROM "patients"


In [24]:
orm_to_df(ormq[:3])

,full_name
0,Cheryl Mason
1,Jason Fleming
2,Tiffany McDonald


## Easy 6
Show first name, last name, and the **full** province name of each patient.

Example: 'Ontario' instead of 'ON'

In [25]:
qstr = """
SELECT
  first_name,
  last_name,
  province_name
FROM Patients p
  INNER JOIN province_names pn ON p.province_id = pn.province_id;
"""
sqlq =  sql_raw(qstr)

In [26]:
# The filter ensures an Inner join is performed
ormq =  Patients.objects.filter(province__isnull=False).\
        values('first_name', 'last_name', province_name=F("province__name"))
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "province_names"."province_name" AS "province_name"
FROM "patients"
INNER JOIN "province_names" ON ("patients"."province_id" = "province_names"."province_id")
WHERE "patients"."province_id" IS NOT NULL


In [27]:
orm_to_df(ormq[:3])

,first_name,last_name,province_name
0,Cheryl,Mason,Ontario
1,Jason,Fleming,British Columbia
2,Tiffany,McDonald,Alberta


## Easy 7
Show how many patients have a birth_date with 2010 as the birth year.
We will do 1995

In [28]:
# sqlq =  sql_raw("SELECT COUNT(*) birth_year FROM Patients WHERE YEAR(birth_date) = 2010;")

# for sqlite (Notice the '1995' instead of 1995)
qstr = """
SELECT COUNT(*) birth_count
FROM Patients
WHERE strftime('%Y', birth_date) = '1995';
"""
sqlq =  sql_raw(qstr)

In [29]:
ormq =  Patients.objects.filter(birth_date__year=1995).\
    annotate(birth_count=Func(F('id'),function='Count')).values('birth_count')
equal(sqlq, ormq)
print_sql(ormq) 

Equal ✔️
SELECT Count("patients"."patient_id") AS "birth_count"
FROM "patients"
WHERE "patients"."birth_date" BETWEEN 1995-01-01 AND 1995-12-31


In [30]:
ormq =  Patients.objects.filter(birth_date__year=1995).aggregate(birth_count=Count('birth_date'))
print(ormq)
equal(sqlq, ormq)

{'birth_count': 5}
Equal ✔️


In [31]:
ormq1 = Patients.objects.filter(birth_date__year=1995).count()
ormq1 = [{'birth_count':ormq1}]
equal(sqlq, ormq1)

Equal ✔️


In [32]:
orm_to_df(ormq)

,birth_count
0,5


## Easy 8
Show how many patients have a birth_date with 2010 as the birth year.
We will do 1995

In [33]:
qstr = """
SELECT
  first_name,
  last_name,
  Max(height) max_height
FROM patients;
"""
sqlq =  sql_raw(qstr)

In [34]:
# Subclassing Max and setting contains_aggregate = False also works
# class NonAggrMax(Max):
#     contains_aggregate = False
    
ormq = Patients.objects.annotate(max_height=Func('height', function='Max'))\
.values('first_name', 'last_name', 'max_height')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       Max("patients"."height") AS "max_height"
FROM "patients"


In [35]:
orm_to_df(ormq)

,first_name,last_name,max_height
0,Cameron,Gupta,190


## Easy 8
Show all columns for patients who have one of the following patient_ids:
1,45,534,879,1000
We will use 1,45,53,87,100

In [36]:
qstr = """
SELECT
  *
FROM patients
WHERE patient_id IN (1, 45, 53, 87, 100);
"""
sqlq =  sql_raw(qstr)

In [37]:
ormq =  Patients.objects.filter(id__in=[1,45,53,87,100]).values(patient_id=F('id'), *common_fields)
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."gender",
       "patients"."birth_date",
       "patients"."city",
       "patients"."province_id",
       "patients"."allergies",
       "patients"."height",
       "patients"."weight",
       "patients"."patient_id" AS "patient_id"
FROM "patients"
WHERE "patients"."patient_id" IN (1,
                                  45,
                                  53,
                                  87,
                                  100)


In [38]:
orm_to_df(ormq)

,first_name,last_name,gender,birth_date,city,province_id,allergies,height,weight,patient_id
0,Cheryl,Mason,F,1978-05-14,Toronto,ON,Peanuts,170,65,1
1,Ella,Lemieux,F,1991-05-01,Halifax,NS,Lactose,164,60,45
2,Avery,Roy,F,1993-05-08,Halifax,NS,None,165,61,53
3,Avery,Chen,F,1989-12-07,Winnipeg,MB,None,163,58,87
4,Nathan,Gupta,M,1986-01-14,Victoria,BC,Lactose,184,82,100


## Easy 9
Show the total number of admissions

In [39]:
sqlq =  sql_raw("SELECT COUNT(*) total FROM admissions;")
ormq =  Admissions.objects.aggregate(total=Count('id'))
equal(sqlq, ormq)

Equal ✔️


In [40]:
ormq1 =  Admissions.objects.count()
ormq1 = [{'total':ormq1}]
equal(sqlq, ormq1)

Equal ✔️


In [41]:
orm_to_df(ormq)

,total
0,170


## Easy 10
Show all the columns from admissions where the patient was admitted and discharged on the same day.

In [46]:
qstr = """
SELECT *
FROM admissions
WHERE admission_date = discharge_date;
"""
sqlq =  sql_raw(qstr)

In [47]:
ormq =  Admissions.objects.filter(admission_date=F('discharge_date')).values("admission_date").annotate(cnt=Count("admission_date")).filter(cnt__gte=2)
# equal(sqlq, ormq)
print_sql(ormq)

SELECT "admissions"."admission_date",
       COUNT("admissions"."admission_date") AS "cnt"
FROM "admissions"
WHERE "admissions"."admission_date" = ("admissions"."discharge_date")
GROUP BY "admissions"."admission_date"
HAVING COUNT("admissions"."admission_date") >= 2


In [48]:
orm_to_df(ormq[:3])

""


## Easy 10
Show the patient id and the total number of admissions for patient_id 579.
We will do 57

In [49]:
qstr = """
SELECT
  patient_id,
  Count(patient_id) admissions
FROM admissions
WHERE patient_id = 57;
"""
sqlq =  sql_raw(qstr)

In [50]:
ormq =  Admissions.objects.filter(patient_id=57)\
        .annotate(admissions=Func(F('patient_id'), function='Count')).values('patient_id', 'admissions')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "admissions"."patient_id",
       Count("admissions"."patient_id") AS "admissions"
FROM "admissions"
WHERE "admissions"."patient_id" = 57


In [51]:
orm_to_df(ormq[:3])

,patient_id,admissions
0,57,2


## Easy 11
Based on the cities that our patients live in, show unique cities that are in province_id 'NS'?

In [52]:
qstr = """
SELECT DISTINCT city
FROM patients
WHERE province_id = 'NS';
"""
sqlq =  sql_raw(qstr)

In [53]:
ormq =  Patients.objects.filter(province__id='NS').values('city').distinct()
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT DISTINCT "patients"."city"
FROM "patients"
WHERE "patients"."province_id" = NS


In [54]:
orm_to_df(ormq)

,city
0,Halifax


## Easy 11
Write a query to find the first_name, last name and birth date of patients who has height greater than 160 and weight greater than 70

In [55]:
qstr = """
SELECT
  first_name,
  last_name,
  birth_date
FROM patients
WHERE height > 160 AND weight > 70;

"""
sqlq =  sql_raw(qstr)

In [56]:
ormq =  Patients.objects.filter(height__gt=160, weight__gt=70).values('first_name', 'last_name', 'birth_date')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."birth_date"
FROM "patients"
WHERE ("patients"."height" > 160
       AND "patients"."weight" > 70)


In [57]:
orm_to_df(ormq[:3])

,first_name,last_name,birth_date
0,Jason,Fleming,1990-08-02
1,Alex,Carter,1976-11-18
2,Gavin,Bryant,1980-02-06


## Easy 12
Write a query to find list of patients first_name, last_name, and allergies from Hamilton where allergies are not null.
We will use Toronto

In [58]:
qstr = """
SELECT
  first_name,
  last_name,
  allergies
FROM patients
WHERE
  city = 'Toronto'
  and allergies IS NOT NULL;
"""
sqlq =  sql_raw(qstr)

In [59]:
ormq =  Patients.objects.filter(city='Toronto', allergies__isnull=False).\
        values('first_name', 'last_name', 'allergies')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."allergies"
FROM "patients"
WHERE ("patients"."allergies" IS NOT NULL
       AND "patients"."city" = Toronto)


In [60]:
ormq1 =  Patients.objects.filter( ~Q(allergies=None), city='Toronto',).\
        values('first_name', 'last_name', 'allergies')
equal(sqlq, ormq1)
print_sql(ormq1)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."allergies"
FROM "patients"
WHERE (NOT ("patients"."allergies" IS NULL)
       AND "patients"."city" = Toronto)


In [61]:
ormq2 =  Patients.objects.filter(city='Toronto').exclude(allergies=None).\
        values('first_name', 'last_name', 'allergies')
equal(sqlq, ormq2)
print_sql(ormq2)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."allergies"
FROM "patients"
WHERE ("patients"."city" = Toronto
       AND NOT ("patients"."allergies" IS NULL))


In [62]:
orm_to_df(ormq[:3])

,first_name,last_name,allergies
0,Cheryl,Mason,Peanuts
1,Kimberly,Lee,Pollen
2,Maria,Singh,Pollen


## Easy 13 
Based on cities where our patient lives in, write a query to display the list of unique city starting with a vowel (a, e, i, o, u). Show the result order in ascending by city.

In [63]:
# You can use city LIKE '[aeiou]%'
qstr = """
SELECT DISTINCT city
FROM patients
WHERE
  city like 'a%'
  or city like 'E%'
  or city like 'I%'
  or city like 'O%'
  or city like 'U%'
ORDER BY city;
"""
sqlq =  sql_raw(qstr)

In [64]:
query = Q()
for ch in ['A','E','I', 'O', 'U']:
    query |= Q(city__istartswith=ch)
ormq =  Patients.objects.filter(query).values('city').distinct().order_by('city')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT DISTINCT "patients"."city"
FROM "patients"
WHERE ("patients"."city" LIKE A% ESCAPE '\' OR "patients"."city" LIKE E% ESCAPE '\' OR "patients"."city" LIKE I% ESCAPE '\' OR "patients"."city" LIKE O% ESCAPE '\' OR "patients"."city" LIKE U% ESCAPE '\')
ORDER BY "patients"."city" ASC


In [65]:
ormq1 =  Patients.objects.filter(city__regex=r"^[AEIOU].+").values('city').distinct().order_by('city')
equal(sqlq, ormq1)
print_sql(ormq1)

Equal ✔️
SELECT DISTINCT "patients"."city"
FROM "patients"
WHERE "patients"."city" REGEXP ^[AEIOU].+
ORDER BY "patients"."city" ASC


In [66]:
orm_to_df(ormq1)

,city
0,Ottawa


## Medium 1
Based on cities where our patient lives in, write a query to display the list of unique city starting with a vowel (a, e, i, o, u). Show the result order in ascending by city.

In [67]:
# Use YEAR(birth_date) for most databases
qstr = """
SELECT
  DISTINCT strftime('%Y', birth_date) birth_year
FROM patients
ORDER BY birth_year;
"""
sqlq =  sql_raw(qstr)

In [68]:
ormq =  Patients.objects.annotate(birth_year=Cast(F('birth_date__year'), CharField()))\
        .distinct().order_by('birth_year').values('birth_year')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT DISTINCT CAST(django_date_extract(year, "patients"."birth_date") AS text) AS "birth_year"
FROM "patients"
ORDER BY "birth_year" ASC


In [69]:
orm_to_df(ormq[:3])

,birth_year
0,1975
1,1976
2,1977


## Medium 2
Show unique first names from the patients table which only occurs once in the list.

For example, if two or more people are named 'John' in the first_name column then don't include their name in the output list. If only 1 person is named 'Leo' then include them in the output.

In [70]:
qstr = """
SELECT first_name
from patients
group by first_name
having count(first_name) = 1
"""
sqlq =  sql_raw(qstr)

In [71]:
ormq =  Patients.objects.values('first_name').annotate(occurance=Count('first_name'))\
        .filter(occurance=1).values('first_name')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name"
FROM "patients"
GROUP BY "patients"."first_name"
HAVING COUNT("patients"."first_name") = 1


## Medium 3
Show patient_id and first_name from patients where their first_name start and ends with 's' and is at least 6 characters long.
We will do start and ends with 'n' and is at least 6 characters long.

In [72]:
qstr = """
SELECT
  patient_id,
  first_name
from patients
where first_name LIKe "n____%n"
"""
sqlq =  sql_raw(qstr)

In [73]:
ormq =  Patients.objects.filter(first_name__regex=r"(?i)^n\w{4,}n$").values('first_name', patient_id=F('id'))
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."patient_id" AS "patient_id"
FROM "patients"
WHERE "patients"."first_name" REGEXP (?i)^n\w{4,}n$


## Medium 4
Show patient_id and first_name from patients where their first_name start and ends with 's' and is at least 6 characters long.
We will do start and ends with 'n' and is at least 6 characters long.

In [74]:
qstr = """
SELECT
  p.patient_id,
  first_name,
  last_name
FROM patients p
  INNER JOIN admissions a on p.patient_id = a.patient_id
WHERE diagnosis = 'Diabetes'
"""
sqlq =  sql_raw(qstr)

In [75]:
ormq = Admissions.objects.filter(diagnosis='Diabetes')\
        .values('patient_id', first_name=F('patient__first_name'), last_name=F('patient__last_name'))
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "admissions"."patient_id",
       "patients"."first_name" AS "first_name",
       "patients"."last_name" AS "last_name"
FROM "admissions"
INNER JOIN "patients" ON ("admissions"."patient_id" = "patients"."patient_id")
WHERE "admissions"."diagnosis" = Diabetes


In [76]:
diabetic_patients = Admissions.objects.filter(diagnosis='Diabetes').values_list('patient_id').distinct()
ormq1 = Patients.objects.filter(id__in=diabetic_patients).values('first_name', 'last_name', patient_id=F('id'))
equal(sqlq, ormq1)
print_sql(ormq1)

Unequal ❌
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."patient_id" AS "patient_id"
FROM "patients"
WHERE "patients"."patient_id" IN
    (SELECT DISTINCT U0."patient_id"
     FROM "admissions" U0
     WHERE U0."diagnosis" = Diabetes)


In [77]:
orm_to_df(ormq[:3])

,patient_id,first_name,last_name
0,6,Gavin,Bryant
1,17,Emily,Tran
2,27,Allison,Rogers


## Medium 5
Display every patient's first_name.
Order the list by the length of each name and then by alphbetically

In [78]:
qstr = """
SELECT
  first_name
from patients p
order by
  LENGTH(first_name),
  first_name;
"""
sqlq =  sql_raw(qstr)

In [79]:
ormq = Patients.objects.order_by(Length('first_name'), 'first_name').values('first_name')
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name"
FROM "patients"
ORDER BY LENGTH("patients"."first_name") ASC, "patients"."first_name" ASC


## Medium 6
Show the total amount of male patients and the total amount of female patients in the patients table.
Display the two results in the same row.

In [80]:
qstr = """
SELECT (
    SELECT count(gender)
    from patients
    where gender = 'M'
  ) male, (
    SELECT count(gender)
    from patients
    where gender = 'F'
  ) female;
"""
sqlq =  sql_raw(qstr)

In [81]:
class NonAggrCount(Count):
    contains_aggregate = False

In [94]:
male_subquery, female_subquery = [
    Patients.objects.filter(gender=g)
    .annotate(**{f"{g}_count": Func(F("gender"), function='Count')})
    .values(f"{g}_count")
    for g in ["M", "F"]
]

# ormq = Patients.objects.annotate(
#     male=Subquery(male_subquery), female=Subquery(female_subquery)
# ).values("male", "female")[:1]
orm = male_subquery | female_subquery
equal(sqlq, ormq)
print_sql(ormq)

TypeError: Merging 'QuerySet' classes must involve the same values in each case.

In [82]:
class NonAggrSum(Sum):
    contains_aggregate = False


ormq = Patients.objects.annotate(
    male=NonAggrSum(Case(When(gender="M", then=1))),
    female=NonAggrSum(Case(When(gender="F", then=1))),
).values("male", "female")
equal(sqlq, ormq)
print_sql(ormq)


Equal ✔️
SELECT SUM(CASE
               WHEN "patients"."gender" = M THEN 1
               ELSE NULL
           END) AS "male",
       SUM(CASE
               WHEN "patients"."gender" = F THEN 1
               ELSE NULL
           END) AS "female"
FROM "patients"


In [83]:
orm_to_df(ormq)

,male,female
0,49,51


## Medium 7
Show first and last name, allergies from patients which have allergies to either 'Penicillin' or 'Morphine'. Show results ordered ascending by allergies then by first_name then by last_name.

In [84]:
qstr = """
select
  first_name,
  last_name,
  allergies
from patients
where
  allergies in ('Penicillin', 'Morphine')
order by
  allergies,
  first_name,
  last_name;
"""
sqlq =  sql_raw(qstr)

In [85]:
ormq = (
    Patients.objects.filter(allergies__in=["Penicillin", "Morphine"])
    .values("first_name", "last_name", "allergies")
    .order_by("allergies", "first_name", "last_name")
)
equal(sqlq, ormq)
print_sql(ormq)


Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       "patients"."allergies"
FROM "patients"
WHERE "patients"."allergies" IN (Penicillin,
                                 Morphine)
ORDER BY "patients"."allergies" ASC,
         "patients"."first_name" ASC,
         "patients"."last_name" ASC


In [86]:
orm_to_df(ormq)

,first_name,last_name,allergies
0,Gavin,Bryant,Penicillin
1,Jeremy,Kim,Penicillin
2,Trevor,Baker,Penicillin


### Medium 8
Show patient_id, diagnosis from admissions. Find patients admitted multiple times for the same diagnosis.

In [87]:
qstr = """
SELECT
  patient_id,
  diagnosis
FROM admissions
GROUP BY
  patient_id,
  diagnosis
HAVING COUNT(*) > 1
"""
sqlq =  sql_raw(qstr)

In [88]:
ormq = (
    Admissions.objects.values("patient_id", "diagnosis")
    .annotate(count=Count("patient_id"))
    .filter(count__gt=1)
    .values("patient_id", "diagnosis")
)

equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "admissions"."patient_id",
       "admissions"."diagnosis"
FROM "admissions"
GROUP BY "admissions"."patient_id",
         "admissions"."diagnosis"
HAVING COUNT("admissions"."patient_id") > 1


In [89]:
orm_to_df(ormq)

""


## Medium 9
Show the city and the total number of patients in the city.
Order from most to least patients and then by city name ascending.

In [90]:
qstr = """
select
  city,
  count(*) population
from patients
group by city 
order by population desc, city
"""
sqlq =  sql_raw(qstr)

In [94]:
ormq = (
    Patients.objects.values("city")
    .annotate(population=Count("id"))
    .order_by("-population", "city")
)

equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."city",
       COUNT("patients"."patient_id") AS "population"
FROM "patients"
GROUP BY "patients"."city"
ORDER BY "population" DESC,
         "patients"."city" ASC


In [95]:
orm_to_df(ormq)

,city,population
0,Calgary,13
1,Montreal,13
2,Toronto,13
3,Vancouver,13
4,Halifax,12
5,Ottawa,12
6,Victoria,12
7,Winnipeg,12


## Medium 10
Show first name, last name and role of every person that is either patient or doctor.
The roles are either "Patient" or "Doctor"

In [103]:
qstr = """
SELECT
  first_name,
  last_name,
  'patient' as Role
FROM patients
UNION all
SELECT
  first_name,
  last_name,
  'doctor' as Role
FROM doctors;
"""
sqlq =  sql_raw(qstr)

In [107]:
ormq = (
    Patients.objects.annotate(role=Value("patient", output_field=CharField()))
    .values("first_name", "last_name", "role")
    .union(
        Doctors.objects.annotate(role=Value("doctor", output_field=CharField())).values(
            "first_name", "last_name", "role"
        ),all=True
    )
)

equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."first_name",
       "patients"."last_name",
       patient AS "role"
FROM "patients"
UNION ALL
SELECT "doctors"."first_name",
       "doctors"."last_name",
       doctor AS "role"
FROM "doctors"


In [108]:
orm_to_df(ormq)

,first_name,last_name,role
0,Cheryl,Mason,patient
1,Jason,Fleming,patient
2,Tiffany,McDonald,patient
3,Alex,Carter,patient
4,Cassandra,Miller,patient
...,...,...,...
115,Elizabeth,Wang,doctor
116,Richard,Lam,doctor
117,Emily,Nguyen,doctor
118,Kevin,Chan,doctor


## Medium 11
Show all allergies and their occurance ordered by occurance. Remove NULL values from query.

In [109]:
qstr = """
SELECT
  allergies,
  count (*) AS total_occurance
FROM patients
WHERE allergies IS NOT null
GROUP BY allergies
ORDER BY total_occurance DESC
"""
sqlq =  sql_raw(qstr)

In [114]:
ormq = (
    Patients.objects.values("allergies")
    .filter(allergies__isnull=False)
    .annotate(total_occurance=Count("allergies"))
    .order_by("-total_occurance")
)

equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "patients"."allergies",
       COUNT("patients"."allergies") AS "total_occurance"
FROM "patients"
WHERE "patients"."allergies" IS NOT NULL
GROUP BY "patients"."allergies"
ORDER BY "total_occurance" DESC
